In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import shutil
import keras
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D,Reshape, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, UpSampling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * (y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1-y_pred), 0, 1)))
    

    f1_val = tp / ( tp + ( (1/2) * (fp+fn) ) + K.epsilon())
    return f1_val

In [21]:
from glob import glob
trn='D:/data/PennA/Penn_Action/*/'
tr= glob(trn)

len(tr)



15

In [22]:
train = []
val = []
test = []
train_y = []
val_y = []
test_y = []

y = 0
for i in tr:
    
    #print(i)
    x = glob(i+'/*/')
    
    #shuffle(x)
    t,tt = train_test_split( x , test_size=0.1, random_state=42)
    t, vv = train_test_split( t , test_size=0.1, random_state=42)
    
    for j in t:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
        
        train.append(j)
        train_y.append(y)
    
    for j in vv:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
            
        val.append(j)
        val_y.append(y)
        
    for j in tt:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
            
        test.append(j)
        test_y.append(y)
        
    y = y+1

from tensorflow.keras.utils import to_categorical
tra_y =  np.array(to_categorical(train_y))
va_y  =  np.array(to_categorical(val_y))
te_y  =  np.array(to_categorical(test_y))

(train, tra_y) = shuffle(train, tra_y)
(val, va_y) = shuffle(val, va_y)
(test, te_y) = shuffle(test, te_y)

In [23]:
len(test)

239

In [24]:

def get_te(k , a) :
    x = glob(k+'/*')
    imgdata=[]
    for i in range(0,15):
        
        a = Image.open(x[i])
        b = a.resize((48, 48))
        c = np.array(b)
        imgdata.append(c.reshape(48, 48,3))
        
    idata = np.array(imgdata)
    X_train = idata
    X_train = X_train.astype('float32') / 255.
    #print(np.shape(X_train))
    return X_train

def get_cat(k) :
    return np.array(k)

In [25]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, filename , labels, batch_size) :
    self.filename = filename
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    y_train = get_cat(batch_y)
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x]), np.array( y_train )

In [26]:
class My_Test_Generator(keras.utils.Sequence) :
  
  def __init__(self, filename , batch_size) :
    self.filename = filename
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x])

In [27]:
batch_size = 16

my_training_batch_generator = My_Custom_Generator(train, tra_y, batch_size)
my_validation_batch_generator = My_Custom_Generator(val, va_y, batch_size)

In [29]:
arr = np.array(My_Test_Generator(test, batch_size).__getitem__(1))
for i in range(2,len(x)):
    x = My_Test_Generator(test, batch_size).__getitem__(i)
    arr = np.concatenate((arr,x),axis=0)

In [30]:
np.shape(arr)

(16, 15, 48, 48, 3)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense

num_frames = 15
frame_height = 48
frame_width = 48
num_channels = 3
num_classes = 15  # Change this to the number of classes in your dataset

model = Sequential()

# CNN
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu'), input_shape=(num_frames, frame_height, frame_width, num_channels)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(8, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))

# LSTM
model.add(LSTM(50))

# Fully connected layer
model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_18 (TimeDi  (None, 15, 46, 46, 64)   1792      
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 15, 23, 23, 64)   0         
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 15, 21, 21, 32)   18464     
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 15, 10, 10, 32)   0         
 stributed)                                                      
                                                                 
 time_distributed_22 (TimeDi  (None, 15, 8, 8, 16)    

In [33]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.9,
    staircase=True)
optimizer = keras.optimizers.Adam(lr_schedule)

In [34]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics='accuracy')
model.fit_generator(generator=my_training_batch_generator, epochs = 100,validation_data = my_validation_batch_generator)

C:\Users\shaif\AppData\Local\Temp\ipykernel_20536\789722107.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=my_training_batch_generator, epochs = 100,validation_data = my_validation_batch_generator)
C:\Users\shaif\AppData\Local\Temp\ipykernel_20536\2168406815.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/100
118/118 [==============================] - 112s 918ms/step - loss: 2.6735 - accuracy: 0.0977 - val_loss: 2.6469 - val_accuracy: 0.1174
Epoch 2/100
118/118 [==============================] - 77s 650ms/step - loss: 2.5594 - accuracy: 0.1238 - val_loss: 2.3768 - val_accuracy: 0.1596
Epoch 3/100
118/118 [==============================] - 86s 729ms/step - loss: 2.4100 - accuracy: 0.1814 - val_loss: 2.3747 - val_accuracy: 0.1878
Epoch 4/100
118/118 [==============================] - 73s 613ms/step - loss: 2.3223 - accuracy: 0.2113 - val_loss: 2.2202 - val_accuracy: 0.2254
Epoch 5/100
118/118 [==============================] - 76s 640ms/step - loss: 2.2589 - accuracy: 0.2332 - val_loss: 2.1910 - val_accuracy: 0.2535
Epoch 6/100
118/118 [==============================] - 70s 587ms/step - loss: 2.1835 - accuracy: 0.2599 - val_loss: 2.1944 - val_accuracy: 0.2723
Epoch 7/100
118/118 [==============================] - 77s 653ms/step - loss: 2.1291 - accuracy: 0.2775 - val_loss: 2.1786 

Epoch 57/100
118/118 [==============================] - 79s 671ms/step - loss: 0.5392 - accuracy: 0.8202 - val_loss: 2.7637 - val_accuracy: 0.3897
Epoch 58/100
118/118 [==============================] - 81s 687ms/step - loss: 0.5240 - accuracy: 0.8239 - val_loss: 2.8153 - val_accuracy: 0.3944
Epoch 59/100
118/118 [==============================] - 79s 673ms/step - loss: 0.5985 - accuracy: 0.7940 - val_loss: 2.6452 - val_accuracy: 0.3756
Epoch 60/100
118/118 [==============================] - 77s 649ms/step - loss: 0.5464 - accuracy: 0.8175 - val_loss: 2.6878 - val_accuracy: 0.4131
Epoch 61/100
118/118 [==============================] - 79s 670ms/step - loss: 0.5194 - accuracy: 0.8287 - val_loss: 2.8582 - val_accuracy: 0.3897
Epoch 62/100
118/118 [==============================] - 80s 677ms/step - loss: 0.5124 - accuracy: 0.8314 - val_loss: 2.6803 - val_accuracy: 0.4038
Epoch 63/100
 20/118 [====>.........................] - ETA: 55s - loss: 0.3965 - accuracy: 0.8660

In [55]:
predictions = model.predict_generator(My_Test_Generator(test, batch_size), verbose=0)
from numpy import array
from numpy import argmax
from sklearn.metrics import accuracy_score
p = np.round(predictions)
score = accuracy_score(test_y, p)
scoreneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
p = np.argmax(predictions, axis=1)
from sklearn.metrics import accuracy_score
score = accuracy_score(test_y, p)
score

C:\Users\shaif\AppData\Local\Temp\ipykernel_1528\3949629984.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(My_Test_Generator(test, batch_size), verbose=0)
C:\Users\shaif\AppData\Local\Temp\ipykernel_1528\1543328956.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [56]:
lossAndMetrics = model.evaluate(arr, te_y)
lossAndMetrics

ValueError: Data cardinality is ambiguous:
  x sizes: 16
  y sizes: 239
Make sure all arrays contain the same number of samples.

In [13]:
#Patch Encoder with Conv2D ,  LSTM , Pos_Emd

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection =keras.Sequential(
            [
                (layers.Conv2D(4, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                (layers.Conv2D(8, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                TimeDistributed(Flatten()),
                layers.Dense(projection_dim)
            ]) 
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded 

In [14]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models

inputs = layers.Input(shape= (10,40,40,3) )

encoded_patches = (PatchEncoder(10, 32 )) (inputs)

for _ in range(6):
    
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    attention_output = layers.MultiHeadAttention (  num_heads=6, key_dim=32, dropout=0.1 )  (x1, x1)
    
    lstm_output =  LSTM(32,return_sequences=True,dropout=0.3)(x1)

    x2 = layers.Add()([attention_output, encoded_patches,lstm_output])

        # Layer Normalization and MLP
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)


    x3 = keras.Sequential(layers.Dense(units=32, activation=tf.nn.gelu) )(x3)

        # Skip connection
    encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

representation = layers.GlobalAvgPool1D()(representation)

outputs = layers.Dense(units=15, activation="softmax") ( representation)

model = keras.Model(inputs=inputs, outputs=outputs)
#model.summary()

In [15]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics='accuracy')
model.fit_generator(generator=my_training_batch_generator, epochs = 50,validation_data = my_validation_batch_generator)

C:\Users\chowd\AppData\Local\Temp\ipykernel_788\1958852391.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=my_training_batch_generator, epochs = 50,validation_data = my_validation_batch_generator)
C:\Users\chowd\AppData\Local\Temp\ipykernel_788\2168406815.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/50
59/59 [==============================] - 80s 1s/step - loss: 2.7010 - accuracy: 0.0975 - val_loss: 2.5393 - val_accuracy: 0.1495
Epoch 2/50
59/59 [==============================] - 75s 1s/step - loss: 2.4694 - accuracy: 0.1551 - val_loss: 2.3966 - val_accuracy: 0.1916
Epoch 3/50
59/59 [==============================] - 73s 1s/step - loss: 2.3188 - accuracy: 0.2116 - val_loss: 2.2126 - val_accuracy: 0.2383
Epoch 4/50
59/59 [==============================] - 73s 1s/step - loss: 2.1786 - accuracy: 0.2425 - val_loss: 2.0953 - val_accuracy: 0.2850
Epoch 5/50
59/59 [==============================] - 73s 1s/step - loss: 2.0074 - accuracy: 0.3177 - val_loss: 2.0630 - val_accuracy: 0.3131
Epoch 6/50
59/59 [==============================] - 74s 1s/step - loss: 1.8937 - accuracy: 0.3609 - val_loss: 2.0963 - val_accuracy: 0.3551
Epoch 7/50
59/59 [==============================] - 74s 1s/step - loss: 1.7027 - accuracy: 0.4270 - val_loss: 2.1092 - val_accuracy: 0.3598
Epoch 8/50
59/59 [==